In [37]:
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold, StratifiedKFold, cross_val_score
from datetime import datetime
#from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder, minmax_scale, scale, LabelBinarizer
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.feature_selection import SelectFromModel, VarianceThreshold,RFE, f_regression
#from minepy import MINE
from mlxtend.regressor import StackingRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import gc
color = sns.color_palette()
from mlxtend.regressor import StackingRegressor, StackingCVRegressor

import tensorflow as tf
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn import preprocessing

In [38]:
sample=pd.read_csv('/home/dope/data_set/medicine/f_sample_20180204.csv')
train=pd.read_csv('/home/dope/data_set/medicine/f_train_20180204.csv',encoding='gbk')
test=pd.read_csv('/home/dope/data_set/medicine/f_test_a_20180204.csv',encoding='gbk')

In [39]:
train.columns.values

array(['id', 'SNP1', 'SNP2', 'SNP3', 'SNP4', 'SNP5', 'SNP6', 'SNP7',
       'SNP8', 'SNP9', 'SNP10', 'SNP11', 'SNP12', 'SNP13', 'SNP14',
       'SNP15', 'SNP16', 'SNP17', 'SNP18', 'SNP19', 'SNP20', 'SNP21',
       'SNP22', 'SNP23', 'RBP4', '年龄', '孕次', '产次', '身高', '孕前体重', 'BMI分类',
       '孕前BMI', '收缩压', '舒张压', '分娩时', '糖筛孕周', 'VAR00007', 'wbc', 'ALT',
       'AST', 'Cr', 'BUN', 'CHO', 'TG', 'HDLC', 'LDLC', 'ApoA1', 'ApoB',
       'Lpa', 'hsCRP', 'SNP24', 'SNP25', 'SNP26', 'SNP27', 'SNP28',
       'SNP29', 'SNP30', 'SNP31', 'SNP32', 'SNP33', 'SNP34', 'SNP35',
       'SNP36', 'SNP37', 'SNP38', 'DM家族史', 'SNP39', 'SNP40', 'SNP41',
       'SNP42', 'SNP43', 'SNP44', 'SNP45', 'SNP46', 'SNP47', 'SNP48',
       'SNP49', 'SNP50', 'SNP51', 'SNP52', 'SNP53', 'SNP54', 'SNP55',
       'ACEID', 'label'], dtype=object)

# 定义处理数据函数

In [40]:
def my_dropna(df,target_data):
    des=df.describe().ix[0]
    index=0
    for col in df.columns.values:
        if des[index]/len(df[col]) <= 0.81:
            df.drop(col,axis=1,inplace=True)
            target_data.drop(col,axis=1,inplace=True)
        index=index+1

In [41]:
def my_fillna(df):
    df.fillna(df.mean(),inplace=True)

In [42]:
def my_norm(df):
    for col in df.columns.values:
        if df[col].mean() > 3 :
            df[col] = df[col] / (df[col].mean() / 3)
            #for index in range(len(df[col])):
            #   df[col].ix[index] = df[col].ix[index] / ( df[col].mean() / 4 )

In [43]:
def norm(df):
    for j in [col for col in df]:
        t_max=df[j].max()
        t_min=df[j].min()
        down = 1 if (t_max-t_min == 0) else (t_max-t_min)
        df[j]=(df[j]-t_min) / down * 255

# Data Pre

In [44]:
label_train=train['label']
train.drop('label',axis=1,inplace=True)
train.drop('id',axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

In [45]:
my_dropna(train,test)
my_fillna(train)
my_fillna(test)
#my_norm(train)
train=preprocessing.scale(train)
test= preprocessing.scale(test)

/home/dope/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [46]:
y_train=[]
for index , value in enumerate(label_train):
    y_train.append([1,0]) if value == 1 else y_train.append([0,1])

In [47]:
train

array([[ -7.07460600e-02,  -1.11069483e+00,   1.04179778e-15, ...,
          3.08135826e-01,  -7.29479467e-01,  -4.86389039e-01],
       [  1.34417514e+00,   2.95247994e-01,   4.77597648e-01, ...,
         -1.18093435e+00,   1.02233619e+00,  -4.86389039e-01],
       [ -1.48566726e+00,  -1.11069483e+00,   4.77597648e-01, ...,
          3.08135826e-01,  -7.29479467e-01,  -4.86389039e-01],
       ..., 
       [  1.34417514e+00,   2.95247994e-01,   4.77597648e-01, ...,
          3.08135826e-01,  -7.29479467e-01,   1.75992942e+00],
       [  1.34417514e+00,  -1.11069483e+00,  -1.86832241e+00, ...,
          3.08135826e-01,  -7.29479467e-01,  -4.86389039e-01],
       [  1.34417514e+00,   2.95247994e-01,   4.77597648e-01, ...,
          3.08135826e-01,  -7.29479467e-01,   1.75992942e+00]])

for col in train.columns.values:
    print(col + ":" + str(train[col].mean()))

# CNN

In [48]:
INPUT_WIDTH=8
CONV_WIDTH=3
TRAINNING_STEP=1000
IN_CHANNEL_SIZE=1
OUT_CHANNEL_SIZE=32
OUT_CHANNEL_1_SIZE=64
OUT_CHANNEL_2_SIZE=128
OUTPUT_SIZE=2

LEARNING_RATE_BASE = 0.08
LEARNING_RATE_DECAY = 0.99

In [49]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.3)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # stride [1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] = 1
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    # stride [1, x_movement, y_movement, 1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def get_train_data(data,times):
    return_data=[]
    for i in range(10):
        return_data.append(np.array(train.ix[times*10+i]).reshape([8,8,1]))
    return return_data
def get_train_y(label,times):
    return_y=[]
    for i in range(10):
        return_y.append(y_train[times*10+i])
    return return_y

def get_test_data(data):
    return_data=[]
    for i in range(200):
        return_data.append(np.array(train.ix[800+i]).reshape([8,8,1]))
    return return_data

def get_test_y(data):
    return_y=[]
    for i in range(200):
        return_y.append(y_train[800+i])
    return return_y

def get_train_data_trainfuc(data,times):
    return_data=[]
    for i in range(10):
        return_data.append(np.array(data[times*10+i]).reshape([8,8,1]))
    return return_data
def get_train_y_trainfuc(label,times):
    return_y=[]
    for i in range(10):
        return_y.append(label[times*10+i])
    return return_y

def get_test_data_trainfuc(data):
    return_data=[]
    for i in range(200):
        return_data.append(np.array(data[i]).reshape([8,8,1]))
    return return_data

def get_test_y_trainfuc(data):
    return_y=[]
    for i in range(200):
        return_y.append(data[i])
    return return_y

def get_target_data_trainfuc(data):
    return_x=[]
    for i in range(len(data)):
        return_x.append(np.array(data[i]).reshape([8,8,1]))
    return return_x

def cal_acc():
    pass

In [50]:
with tf.variable_scope("cnn",reuse=tf.AUTO_REUSE):
    train_X = tf.placeholder(tf.float32,[None,INPUT_WIDTH,INPUT_WIDTH,1])
    train_y = tf.placeholder(tf.float32,[None,OUTPUT_SIZE])
    keep_prob = tf.placeholder(tf.float32)
    global_step = tf.Variable(0, trainable=False)
    # conv1 layer
    W_conv1 = weight_variable([CONV_WIDTH,CONV_WIDTH,IN_CHANNEL_SIZE,OUT_CHANNEL_SIZE])
    b_conv1 = bias_variable([OUT_CHANNEL_SIZE])
    h_conv1 = tf.nn.tanh(conv2d(train_X,W_conv1)+b_conv1) #output size is 8x8x2
    h_pool1 = max_pool_2x2(h_conv1) #output size is 4x4xOUT_CHANNEL_SIZE
    
    #conv2 layer
    W_conv2 = weight_variable([CONV_WIDTH,CONV_WIDTH,OUT_CHANNEL_SIZE,OUT_CHANNEL_1_SIZE])
    b_conv2 = bias_variable([OUT_CHANNEL_1_SIZE])
    h_conv2 = tf.nn.tanh(conv2d(h_pool1,W_conv2)+b_conv2)#output size is 4x4xOUT_CHANNEL_1_SIZE
    h_pool2 = max_pool_2x2(h_conv2) #output size is 2x2xOUT_CHANNEL_1_SIZE
    
    #conv3 layer
    #W_conv3 = weight_variable([CONV_WIDTH,CONV_WIDTH,OUT_CHANNEL_1_SIZE,OUT_CHANNEL_2_SIZE])
    #b_conv3 = bias_variable([OUT_CHANNEL_2_SIZE])
    #h_conv3 = tf.nn.tanh(conv2d(h_pool2,W_conv3)+b_conv3) #output size is 4x4xOUT_CHANNEL_2_SIZE
    #h_pool3 = max_pool_2x2(h_conv3) #output size is 2*2*OUT_CHANNEL_2_SIZE
    
    #func1 layer
    W_fc1 = weight_variable([4*4*OUT_CHANNEL_1_SIZE,6])
    b_fc1 = bias_variable([6])
    h_pool1_flat = tf.reshape(h_conv2,[-1,4*4*OUT_CHANNEL_1_SIZE])
    h_fc1 = tf.nn.tanh(tf.matmul(h_pool1_flat, W_fc1) + b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    #func2 layer
    W_fc2 = weight_variable([6,2])
    b_fc2 = bias_variable([2])
    prediction = tf.nn.softmax(tf.matmul(h_fc1,W_fc2) + b_fc2)
    
    #ops
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(train_y * tf.log(prediction),reduction_indices=[1]))
    #loss_op = tf.reduce_mean(tf.square(train_y - prediction))
    
    #other param
    learning_rate = tf.train.exponential_decay(
            LEARNING_RATE_BASE,
            global_step,
            200, LEARNING_RATE_DECAY,
            staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cross_entropy,global_step=global_step)
    
    #accuracy
    y_true = tf.argmin(train_y,1)
    y_pred= tf.argmin(prediction,1)
    correct_prediction = tf.equal(tf.argmin(prediction,1), tf.argmin(train_y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for j in range(86):
        for i in range(80):
            _data=get_train_data(train,i)
            _label=get_train_y(y_train,i)
            sess.run(train_op,feed_dict={train_X:_data , train_y:_label , keep_prob:0.3})
        print(j,sess.run(cross_entropy,feed_dict={train_X:[_data[0]] , train_y:[_label[0]] , keep_prob:0.3}))
    #sess.run(train_op,feed_dict={train_X: [np.array(train.ix[0]).reshape([8,8,1]) , np.array(train.ix[1]).reshape([8,8,1])] , train_y:[y_train[0],y_train[1]] , keep_prob:0.3})
    test_data = get_test_data(train)
    test_y = get_test_y(y_train)
    print(sess.run(prediction,feed_dict={train_X:_data , train_y:_label , keep_prob:0.3}))
    print(sess.run(accuracy,feed_dict={train_X:test_data , train_y:test_y , keep_prob:0.3}))
        
    true_y , pred_y = sess.run([y_true , y_pred],feed_dict={train_X:test_data , train_y:test_y , keep_prob:0.3})
    p = precision_score(true_y,pred_y,average='binary')
    r = recall_score(true_y,pred_y,average='binary')
    f1score = f1_score(true_y,pred_y,average='binary')
    print(p,r,f1score)
    #print(W_conv1.eval())

In [51]:
def train_func(train_data,train_label,test_data,test_label):
    with tf.Session() as sess:
        sess.run(init)
        for j in range(86):
            for i in range(int(len(train_data)/10)):
                _data=get_train_data_trainfuc(train_data,i)
                _label=get_train_y_trainfuc(train_label,i)
                sess.run(train_op,feed_dict={train_X:_data , train_y:_label , keep_prob:0.3})
            #print(j,sess.run(cross_entropy,feed_dict={train_X:[_data[0]] , train_y:[_label[0]] , keep_prob:0.3}))
        #sess.run(train_op,feed_dict={train_X: [np.array(train.ix[0]).reshape([8,8,1]) , np.array(train.ix[1]).reshape([8,8,1])] , train_y:[y_train[0],y_train[1]] , keep_prob:0.3})
        test_data_fuc = get_test_data_trainfuc(test_data)
        test_y_fuc = get_test_y_trainfuc(test_label)
        print(sess.run(prediction,feed_dict={train_X:_data , train_y:_label , keep_prob:0.3}))
        print(sess.run(accuracy,feed_dict={train_X:test_data_fuc , train_y:test_y_fuc , keep_prob:0.3}))
            
        true_y , pred_y = sess.run([y_true , y_pred],feed_dict={train_X:test_data_fuc , train_y:test_y_fuc })
        p = precision_score(true_y,pred_y,average='binary')
        r = recall_score(true_y,pred_y,average='binary')
        f1score = f1_score(true_y,pred_y,average='binary')
        print(p,r,f1score)
        
        gene_target=get_target_data_trainfuc(test)
        target = sess.run([y_pred],feed_dict={train_X:gene_target})
        target=np.array(target).reshape((200))
        target_csv = pd.DataFrame(columns=['result'],data=target)
        target_csv.to_csv('./submission/submission'+str(f1score)+'.csv',index=False)
        #print(W_conv1.eval())

In [52]:
np.concatenate((train[0:0],train[200:])).shape

(800, 64)

In [53]:
for index in range(5):
    test_data=train[index*200:index*200+200]
    test_label=y_train[index*200:index*200+200]
    train_data=np.concatenate((train[0:index*200],train[index*200+200:]))
    train_label=y_train[0:index*200]+y_train[index*200+200:]
    train_func(train_data,train_label,test_data,test_label)

[[  9.98813272e-01   1.18679611e-03]
 [  9.99981403e-01   1.86347679e-05]
 [  9.48484898e-01   5.15151508e-02]
 [  2.85156100e-04   9.99714792e-01]
 [  9.99977827e-01   2.21496939e-05]
 [  9.99881625e-01   1.18409349e-04]
 [  8.44564319e-01   1.55435696e-01]
 [  9.99983311e-01   1.66679110e-05]
 [  9.90584910e-01   9.41510592e-03]
 [  9.97151196e-01   2.84885964e-03]]
0.65
0.638554216867 0.569892473118 0.602272727273
[[  9.97906327e-01   2.09368905e-03]
 [  9.97593105e-01   2.40692217e-03]
 [  9.96760666e-01   3.23941512e-03]
 [  8.14419065e-04   9.99185622e-01]
 [  9.97911394e-01   2.08861334e-03]
 [  9.97377872e-01   2.62215175e-03]
 [  9.99668121e-01   3.31861200e-04]
 [  9.97910559e-01   2.08942220e-03]
 [  9.97324467e-01   2.67547066e-03]
 [  9.50540125e-01   4.94598709e-02]]
0.645
0.730769230769 0.532710280374 0.616216216216
[[  9.98396814e-01   1.60322350e-03]
 [  9.98398125e-01   1.60181429e-03]
 [  9.11517620e-01   8.84823948e-02]
 [  6.36430748e-04   9.99363601e-01]
 [  9.983

In [54]:
#y_train